In [1]:
import numpy as np

# Part 1

In [2]:
test = np.array(['0,9 -> 5,9',
                 '8,0 -> 0,8',
                 '9,4 -> 3,4',
                 '2,2 -> 2,1',
                 '7,0 -> 7,4',
                 '6,4 -> 2,0',
                 '0,9 -> 2,9',
                 '3,4 -> 1,4',
                 '0,0 -> 8,8',
                 '5,5 -> 8,2'])

In [3]:
def line_intersection(line0, line1):
    #We can use the fact we know intersections will be at integer coordinates to help
    p, p1 = line0
    r = p1-p
    
    q, q1 = line1
    s = q1-q
    
    #Values for calculating intersection
    qps = np.cross((q - p), s)
    qpr = np.cross((q - p), r)
    rs = np.cross(r, s)
    
    overlap = []
    
    #Colinear
    if rs == 0 and qpr == 0:
        #range along line0 for colinear overlap
        t0 = np.dot((q - p), r) / np.dot(r, r)
        t1 = np.dot((q + s - p), r) / np.dot(r, r)
        
        #If lines are colinear but do not overlap, return
        if (t0 < 0. and t1 < 0.) or (t0 > 1. and t1 > 1.):
            return np.array(overlap)
        
        #Make sure t0 is smaller than t1
        if t1 < t0:
            t2 = t1
            t1 = t0
            t0 = t2
            
        #Make sure t0 and t1 are inside line0
        if t0 < 0:
            t0 = 0.0
        if t1 > 1:
            t1 = 1.0
        
        #If t0 == t1 then the overlap is at one endor the other, so pick t0
        if t0 == t1:
            pq = p + (t0*r)
            #check if integer. Use round to catch floating point errors
            if round(pq[0],3).is_integer() and round(pq[0],3).is_integer():
                overlap.append(np.around(pq).astype(int))
            return np.array(overlap)
            
        #Find range that (should) tests integer positions
        start = p+(t0*r)
        stop = p+(t1*r)
        delta = np.abs(start-stop)
        t_delt = (t1-t0)/np.max(delta)
            
        for t in np.arange(t0, t1+(t_delt/2), t_delt):
            pq = p + (t*r)
            #check if integer. Use round to catch floating point errors
            if round(pq[0],3).is_integer() and round(pq[0],3).is_integer():
                pq = np.around(pq).astype(int)
                if len(overlap) == 0:
                    overlap.append(pq)
                elif np.equal(overlap, pq).all(1).any() == False:
                    overlap.append(pq)
            
    #Intersection
    elif rs != 0:
        t = qps / rs
        u = qpr / rs
        #Make sure intersection point is on both lines
        if (t >= 0 and t <= 1) and (u >= 0 and u <= 1):
            pq = p + (t*r)
            if round(pq[0],3).is_integer() and round(pq[0],3).is_integer():
                overlap.append(np.around(pq).astype(int))
        
    return np.array(overlap)

def get_lines(data):
    #Convert input into line start/end coordinates
    lines = []
    for line in data:
        p_s, q_s = line.split(' -> ')
        p_s = p_s.split(',')
        p_s = np.array(p_s).astype(int)
        q_s = q_s.split(',')
        q_s = np.array(q_s).astype(int)
        lines.append([p_s, q_s])
    lines = np.array(lines)
    return lines

def keep_horizontal_verticle(lines):
    #Only keep horizontal orverticle lines
    delete = []
    for i in range(0, len(lines)):
        if lines[i][0,0] != lines[i][1,0] and lines[i][0,1] != lines[i][1,1]:
            delete.append(i)
    lines = np.delete(lines, delete, 0)
    return lines

def count_intersections(data, xy=True):
    lines = get_lines(data)
    if xy:
        lines = keep_horizontal_verticle(lines)
    
    #For every line, check if it intersects the following lines
    cross = {}
    for i in range(0, len(lines)-1):
        for j in range(i+1, len(lines)):
            intct = line_intersection(lines[i], lines[j])
            if len(intct) > 0:
                for c in intct:
                    if tuple(c) not in cross.keys():
                        cross[tuple(c)] = 1
                    else:
                        cross[tuple(c)] +=1
    return len(cross.keys())

In [4]:
print(count_intersections(test))

5


In [5]:
inpt = np.genfromtxt('day5_input.txt', dtype=str, delimiter='\n')
print('Part 1 Result:', count_intersections(inpt))

Part 1 Result: 7438


# Part 2

In [6]:
print(count_intersections(test, False))

12


In [7]:
print('Part 2 Result:', count_intersections(inpt, False))

Part 2 Result: 21406
